## Naudojamos bibliotekos

In [2]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import itertools
from fuzzywuzzy import process, fuzz

c:\users\ruta\desktop\studijos\4 semestras\02. python\venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Nuskaitomi duomenys

In [3]:
data = pd.read_csv('data.csv')

## Sukuriamos autorių pavardžių ir inicialų kombinacijos ir išsaugomos Excel failuose

#### Gautos autorių pavardės išsaudomos Excel failuose, kad būtų galima jas peržiūrėti

##### autoriai1: Pavarde, V.
##### autoriai2: Pavarde Pavarde, V. - pasalinti su zodeliu ir
##### autoriai3: Pavarde-Pavarde, V.
##### autoriai4: Pavarde Pavarde Pavarde, V. - pasalinti dauguma reiksmiu
##### autoriai5: Pavarde, V.V.
##### autoriai6: Pavarde, V.V.V.
##### autoriai7: Pavarde, V.V.V.V.
##### autoriai8: Pavarde, V.V.V.V.V.

In [4]:
# autoriai1: Pavarde, V.
autoriai_parser = re.compile("([A-Ž][a-ž]+, [A-Ž].)")
autoriai1 = pd.DataFrame(data.projection_apa.map(lambda x: re.findall(autoriai_parser, x)))
#autoriai1.to_excel('autoriai1.xlsx')

In [5]:
# autoriai2: Pavarde Pavarde, V.
autoriai_parser2 = re.compile("([A-Ž][a-ž]+ [A-Ž][a-ž]+, [A-Ž].)")
autoriai2 = pd.DataFrame(data.projection_apa.map(lambda x:  re.findall(autoriai_parser2, x)))
#autoriai2.to_excel('autoriai2.xlsx')

In [6]:
# autoriai3: Pavarde-Pavarde, V.
autoriai_parser3 = re.compile("([A-Ž][a-ž]+[-][A-Ž][a-ž]+, [A-Ž].)")
autoriai3 = pd.DataFrame(data.projection_apa.map(lambda x:  re.findall(autoriai_parser3, x)))
#autoriai3.to_excel('autoriai3.xlsx')

In [7]:
# autoriai4: Pavarde Pavarde Pavarde, V.
autoriai_parser4 = re.compile("([A-Ž][a-ž]+ [A-Ž][a-ž]+ [A-Ž][a-ž]+, [A-Ž].)")
autoriai4 = pd.DataFrame(data.projection_apa.map(lambda x:  re.findall(autoriai_parser4, x)))
#autoriai4.to_excel('autoriai4.xlsx')

In [8]:
# autoriai5: Pavarde, V.V.
autoriai_parser5 = re.compile("([A-Ž][a-ž]+, [A-Ž][.]+ [A-Ž][.])")
autoriai5 = pd.DataFrame(data.projection_apa.map(lambda x:
                                                 re.findall(autoriai_parser5, x)))
#autoriai5.to_excel('autoriai5.xlsx')

In [9]:
# autoriai6: Pavarde, V.V.V.
autoriai_parser6 = re.compile("([A-Ž][a-ž]+, [A-Ž][.]+ [A-Ž][.]+ [A-Ž][.])")
autoriai6 = pd.DataFrame(data.projection_apa.map(lambda x:  re.findall(autoriai_parser6, x)))
#autoriai6.to_excel('autoriai6.xlsx')

In [10]:
# autoriai7: Pavarde, V.V.V.V.
autoriai_parser7 = re.compile("([A-Ž][a-ž]+, [A-Ž][.]+ [A-Ž][.]+ [A-Ž][.]+ [A-Ž][.])")
autoriai7 = pd.DataFrame(data.projection_apa.map(lambda x:  re.findall(autoriai_parser7, x)))
#autoriai7.to_excel('autoriai7.xlsx')

In [11]:
# autoriai8: Pavarde, V.V.V.V.V.
autoriai_parser8 = re.compile("([A-Ž][a-ž]+, [A-Ž][.]+ [A-Ž][.]+ [A-Ž][.]+ [A-Ž][.]+ [A-Ž][.])")
autoriai8 = pd.DataFrame(data.projection_apa.map(lambda x:  re.findall(autoriai_parser8, x)))
# autoriai8.to_excel('autoriai8.xlsx')

## Pašalinami publikacijų metai iš publikacijos aprašymo (metai yra atskiras stulpelis)

In [12]:
data1 = data

data1['new']=""

for index, row in enumerate(data1.projection_apa):
    autoriai_parsed = re.sub("([(][0-9][0-9][0-9][0-9][)])", "", row)
    data1.new[index] = autoriai_parsed

C:\Users\Ruta\AppData\Local\Temp\ipykernel_5464\1990771342.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.new[index] = autoriai_parsed


## doi adreso pašalinimas iš publikacijos aprašymo

In [13]:
doi = data1['new'].str.split('doi:', expand=True)

In [14]:
data1['doi_address'] = doi[1]

In [15]:
data1['doi'] = pd.notnull(data1['doi_address']).astype(int)

In [16]:
data1 = data1.drop(['doi_address'], axis=1)

In [17]:
data1['new_1'] = doi[0]

## "Retrieved" adreso pašalinimas iš publikacijos aprašymo

In [18]:
Retrieved = data1['new_1'].str.split('Retrieved ', expand=True)

In [19]:
data1['new_2'] = Retrieved[0]

## "Žiūrėta" adreso pašalinimas iš publikacijos aprašymo

In [20]:
ziureta = data1['new_2'].str.split(' Žiūrėta ', expand=True)

In [21]:
data1['new_3'] = ziureta[0]

In [22]:
data1 = data1.drop(['new', 'new_1', 'new_2'], axis=1)

## Išskiriami publikacijų puslapiai ir pašalinami iš publikacijos aprašymo

#### Pirmiausia randamos ir pašalinamos skaičių kombinacijos, kurios yra nerealistinės (klaidingi įrašai arba išskirtys). Jie nebus naudojami puslapių skaičiui aprašyti.

In [23]:
data1['new_4']=""

for index, row in enumerate(data1.new_3):
    autoriai_parsed = re.sub("([0-9]+[-]+[0-9][0-9][0-9][0-9][0-9][0-9])", "", row)
    data1.new_4[index] = autoriai_parsed

C:\Users\Ruta\AppData\Local\Temp\ipykernel_5464\3223298426.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.new_4[index] = autoriai_parsed


#### Tuomet randami ir pašalinami publikacijos puslapiai, kurie yra ypač didelių verčių. Dalis jų yra nerealistiniai, dėl to tokios vertės bus šalinamos

In [24]:
puslapiai_parser2 = re.compile("([0-9]+[-]+[0-9][0-9][0-9][0-9][0-9])")
puslapiai2 = pd.DataFrame(data1.new_4.map(lambda x:  re.findall(puslapiai_parser2, x)))

In [25]:
puslapiai2.rename(columns = {'new_4' : 'Puslapiai_5_zenklai'}, inplace = True)

In [26]:
data1['Puslapiai_5_zenklai'] = puslapiai2['Puslapiai_5_zenklai']

In [27]:
data1['new_5']=""

for index, row in enumerate(data1.new_4):
    autoriai_parsed = re.sub("([0-9]+[-]+[0-9][0-9][0-9][0-9][0-9])", "", row)
    data1.new_5[index] = autoriai_parsed

C:\Users\Ruta\AppData\Local\Temp\ipykernel_5464\3682309785.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.new_5[index] = autoriai_parsed


In [28]:
data1 = data1.drop(['new_3', 'new_4'], axis=1)

#### Atskiriami ir pašalinami publikacijos puslapiai, kurie yra keturženkliai

In [29]:
puslapiai_parser3 = re.compile("([0-9]+[-]+[0-9][0-9][0-9][0-9])")
puslapiai3 = pd.DataFrame(data1.new_5.map(lambda x:  re.findall(puslapiai_parser3, x)))

In [30]:
puslapiai3.rename(columns = {'new_5' : 'Puslapiai_4_zenklai'}, inplace = True)

In [31]:
data1['Puslapiai_4_zenklai'] = puslapiai3['Puslapiai_4_zenklai']

In [32]:
data1['new_6']=""

for index, row in enumerate(data1.new_5):
    autoriai_parsed = re.sub("([0-9]+[-]+[0-9][0-9][0-9][0-9])", "", row)
    data1.new_6[index] = autoriai_parsed

C:\Users\Ruta\AppData\Local\Temp\ipykernel_5464\3752697549.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.new_6[index] = autoriai_parsed


#### Atskiriami ir pašalinami publikacijos puslapiai, kurie yra triženkliai

In [33]:
puslapiai_parser4 = re.compile("([0-9]+[-]+[0-9][0-9][0-9])")
puslapiai4 = pd.DataFrame(data1.new_6.map(lambda x:
                                          re.findall(puslapiai_parser4, x)))

In [34]:
puslapiai4.rename(columns = {'new_6' : 'Puslapiai_3_zenklai'}, inplace = True)

In [35]:
data1['Puslapiai_3_zenklai'] = puslapiai4['Puslapiai_3_zenklai']

In [36]:
data1['new_7']=""

for index, row in enumerate(data1.new_6):
    autoriai_parsed = re.sub("([0-9]+[-]+[0-9][0-9][0-9])", "", row)
    data1.new_7[index] = autoriai_parsed

C:\Users\Ruta\AppData\Local\Temp\ipykernel_5464\3381633236.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.new_7[index] = autoriai_parsed


In [37]:
data1 = data1.drop(['new_5', 'new_6'], axis=1)

#### Atskiriami ir pašalinami publikacijos puslapiai, kurie yra dviženkliai

In [38]:
puslapiai_parser5 = re.compile("([0-9]+[-]+[0-9][0-9])")
puslapiai5 = pd.DataFrame(data1.new_7.map(lambda x:  re.findall(puslapiai_parser5, x)))

In [39]:
puslapiai5.rename(columns = {'new_7' : 'Puslapiai_2_zenklai'}, inplace = True)

In [40]:
data1['Puslapiai_2_zenklai'] = puslapiai5['Puslapiai_2_zenklai']

In [41]:
data1['new_8']=""

for index, row in enumerate(data1.new_7):
    autoriai_parsed = re.sub("([0-9]+[-]+[0-9][0-9])", "", row)
    data1.new_8[index] = autoriai_parsed

C:\Users\Ruta\AppData\Local\Temp\ipykernel_5464\3399269904.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.new_8[index] = autoriai_parsed


In [42]:
data1 = data1.drop(['new_7'], axis=1)

## Autorių ištrynimas. Autorius padedama trinti nuo didžiausias išskirtis turinčių kombinacijų, kad būtų pašalintos visos pavardės ir inicialai

#### Ištrinami autoriai8: Pavarde, V.V.V.V.V.

In [43]:
data1['new_9']=""

for index, row in enumerate(data1.new_8):
    autoriai_parsed = re.sub("([A-Ž][a-ž]+, [A-Ž][.]+ [A-Ž][.]+ [A-Ž][.]+ [A-Ž][.]+ [A-Ž][.])", "", row)
    data1.new_9[index] = autoriai_parsed

C:\Users\Ruta\AppData\Local\Temp\ipykernel_5464\3232666677.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.new_9[index] = autoriai_parsed


#### Ištrinami autoriai7: Pavarde, V.V.V.V.

In [44]:
data1['new_10']=""

for index, row in enumerate(data1.new_9):
    autoriai_parsed = re.sub("([A-Ž][a-ž]+, [A-Ž][.]+ [A-Ž][.]+ [A-Ž][.]+ [A-Ž][.])", "", row)
    data1.new_10[index] = autoriai_parsed

C:\Users\Ruta\AppData\Local\Temp\ipykernel_5464\1318028907.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.new_10[index] = autoriai_parsed


#### Ištrinami autoriai6: Pavarde, V.V.V.

In [45]:
data1['new_11']=""

for index, row in enumerate(data1.new_10):
    autoriai_parsed = re.sub("([A-Ž][a-ž]+, [A-Ž][.]+ [A-Ž][.]+ [A-Ž][.])", "", row)
    data1.new_11[index] = autoriai_parsed

C:\Users\Ruta\AppData\Local\Temp\ipykernel_5464\1310412160.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.new_11[index] = autoriai_parsed


#### Ištrinami autoriai5: Pavarde, V.V.

In [46]:
data1['new_12']=""

for index, row in enumerate(data1.new_11):
    autoriai_parsed = re.sub("([A-Ž][a-ž]+, [A-Ž][.]+ [A-Ž][.])", "", row)
    data1.new_12[index] = autoriai_parsed

C:\Users\Ruta\AppData\Local\Temp\ipykernel_5464\2687834503.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.new_12[index] = autoriai_parsed


#### Ištrinami autoriai4: Pavarde Pavarde Pavarde, V.

In [47]:
data1['new_13']=""

for index, row in enumerate(data1.new_12):
    autoriai_parsed = re.sub("([A-Ž][a-ž]+ [A-Ž][a-ž]+ [A-Ž][a-ž]+, [A-Ž].)", "", row)
    data1.new_13[index] = autoriai_parsed

C:\Users\Ruta\AppData\Local\Temp\ipykernel_5464\1542125854.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.new_13[index] = autoriai_parsed


#### Ištrinami autoriai3: Pavarde-Pavarde, V.

In [48]:
data1['new_14']=""

for index, row in enumerate(data1.new_13):
    autoriai_parsed = re.sub("([A-Ž][a-ž]+[-][A-Ž][a-ž]+, [A-Ž].)", "", row)
    data1.new_14[index] = autoriai_parsed

C:\Users\Ruta\AppData\Local\Temp\ipykernel_5464\1148506018.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.new_14[index] = autoriai_parsed


#### Ištrinami autoriai2: Pavarde Pavarde, V.

In [49]:
data1['new_15']=""

for index, row in enumerate(data1.new_14):
    autoriai_parsed = re.sub("([A-Ž][a-ž]+ [A-Ž][a-ž]+, [A-Ž].)", "", row)
    data1.new_15[index] = autoriai_parsed

C:\Users\Ruta\AppData\Local\Temp\ipykernel_5464\384823790.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.new_15[index] = autoriai_parsed


#### Ištrinami autoriai1: Pavarde, V.

In [50]:
data1['new_16']=""

for index, row in enumerate(data1.new_15):
    autoriai_parsed = re.sub("([A-Ž][a-ž]+, [A-Ž].)", "", row)
    data1.new_16[index] = autoriai_parsed

C:\Users\Ruta\AppData\Local\Temp\ipykernel_5464\3033139105.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.new_16[index] = autoriai_parsed


In [51]:
data1 = data1.drop(['new_8', 'new_9', 'new_10', 'new_11', 'new_12', 'new_13', 'new_14', 'new_15'], axis=1)

## Publikacijos pavadinimo atskyrimas

In [52]:
Publikacija = data1['new_16'].str.split(' <i><i>', expand=True)

In [53]:
data1['publication'] = Publikacija[0]

In [54]:
data1['new_17'] = Publikacija[1]

In [55]:
data1 = data1.drop(['new_16'], axis=1)

## Leidinio numerio atskyrimas

#### Atskiriami numeriai, kurie turi 2 identifikatorius

In [56]:
Leidinio_numeriai_2 = data1['new_17'].str.split('</i></i>, <i>', expand=True)

In [57]:
data1['Leidinio_numeriai_2'] = Leidinio_numeriai_2[1]

In [58]:
data1['new_17'] = Leidinio_numeriai_2[0]

##### Atskiriamas leidinio numeris nuo mėnesinio numerio

In [59]:
Leidinio_numeris = data1['Leidinio_numeriai_2'].str.split('</i>', expand=True)

In [60]:
data1['journal_number'] = Leidinio_numeris[0]

##### Atskiriami leidinio puslapiai, kurie buvo užrašyti kitaip, nei anksčiau atskirtieji
##### Mėnesiniai leidinio numeriai nesaugomi, nes formatas per daug nevienodas ir šių duomenų informacija yra neauktuali

In [61]:
puslapiai = Leidinio_numeris[1].str.split(', ', expand=True)

In [62]:
puslapiai_1 = puslapiai[1].str.split('.', expand=True)

In [63]:
data1['Puslapiai'] = puslapiai_1[0]

#### Atskiriami numeriai, kurie turi 1 identifikatorių

In [64]:
Leidinio_numeriai_1 = data1['new_17'].str.split('</i></i>, ', expand=True)

In [65]:
data1['Leidinio_numeriai_1'] = Leidinio_numeriai_1[1]

In [66]:
data1['new_17'] = Leidinio_numeriai_1[0]

##### Atskiriami leidinio puslapiai, kurie buvo užrašyti kitaip, nei anksčiau atskirtieji

In [67]:
puslapiai_2 = data1['Leidinio_numeriai_1'].str.split(', ', expand=True)

In [68]:
Leidinio_numeris_1 = puslapiai_2[0].str.split('.', expand=True)

In [69]:
puslapiai_2_2 = puslapiai_2[1].str.split('.', expand=True)

In [70]:
data1['Leidinio_numeris_1'] = Leidinio_numeris_1[0]

In [71]:
data1['Puslapiai_1'] = puslapiai_2_2[0]

In [72]:
data1 = data1.drop(['Leidinio_numeriai_1', 'Leidinio_numeriai_2'], axis=1)

## Leidinio pavadinimo išvalymas

####  [elektroninis išteklius] prierašo pašalinimas

In [73]:
elektroninis_isteklius = data1['new_17'].str.split(' \[Elektroninis išteklius\]', expand=True)

In [74]:
data1['new_17'] = elektroninis_isteklius[0]

In [75]:
elektroninis_isteklius_1 = data1['new_17'].str.split(' \[elektroninis išteklius\]', expand=True)

In [76]:
data1['new_17'] = elektroninis_isteklius_1[0]

####  \</i>\</i> prierašo pašalinimas

In [77]:
prierasas = data1['new_17'].str.split('</i></i>', expand=True)

In [78]:
data1['new_17'] = prierasas[0]

## Atskiriami leidinių pavadinimai kitomis kalbomis

In [79]:
Leidinio_pavadinimas_uzsienio_kalba = data1['new_17'].str.split(' = ', expand=True)

In [80]:
data1['journal_name'] = Leidinio_pavadinimas_uzsienio_kalba[0]

In [81]:
data1['journal_name_in_foreign_language_1'] = Leidinio_pavadinimas_uzsienio_kalba[1]

In [82]:
data1['journal_name_in_foreign_language_2'] = Leidinio_pavadinimas_uzsienio_kalba[2]

In [83]:
data1['journal_name_in_foreign_language_3'] = Leidinio_pavadinimas_uzsienio_kalba[3]

In [84]:
data1 = data1.drop(['new_17'], axis=1)

## Puslapių sutvarkymas

### Tušti sąrašai pakeičiami į NaN reikšmes

In [85]:
sarasas = ['Puslapiai_5_zenklai', 'Puslapiai_4_zenklai', 'Puslapiai_3_zenklai', 'Puslapiai_2_zenklai']

In [86]:
for column in sarasas:
    data1[column] = data1[column].apply(lambda x: np.nan if len(x)==0 else x)

### Sukuriamas vienas puslapių stulpelis

In [87]:
data1["Puslapiai_5_zenklai"] = data1["Puslapiai_5_zenklai"].fillna(data1["Puslapiai_4_zenklai"])

In [88]:
data1["Puslapiai_5_zenklai"] = data1["Puslapiai_5_zenklai"].fillna(data1["Puslapiai_3_zenklai"])

In [89]:
data1["Puslapiai_5_zenklai"] = data1["Puslapiai_5_zenklai"].fillna(data1["Puslapiai_2_zenklai"])

In [90]:
data1["Puslapiai_5_zenklai"] = data1["Puslapiai_5_zenklai"].fillna(data1["Puslapiai"])

In [91]:
data1["Puslapiai_5_zenklai"] = data1["Puslapiai_5_zenklai"].fillna(data1["Puslapiai_1"])

In [92]:
data1 = data1.drop(['Puslapiai_4_zenklai', 'Puslapiai_3_zenklai', "Puslapiai_2_zenklai", "Puslapiai", "Puslapiai_1"], axis=1)

In [93]:
data1.rename(columns = {'Puslapiai_5_zenklai' : 'pages'}, inplace = True)

### Puslapiai iš listo keičiami į stringą. Sudaromi 2 stulpeliai: puslapiai ir publikacijos apimtis

#### Puslapių stulpelyje yra NaN, str ir list tipo reikšmių. Pirma užpildomos Nan reikšmės su 0, tuomet parašoma funkcija, kuri tikrina, ar puslapio formatas yra tinkamas (turi brūkšnį, kas reiškia, kad yra pradžios ir pabaigos puslapiai). Jei reikšmė tinkama, tikrinama, ar ji yra list, ar str. Atitinkamai vertė paverčiama į int. Apskaičiuojamas puslapių pradžios ir pabaigos skirtumas ir gaunama publikacijos apimtis puslapiais

In [94]:
data1['pages'] = data1['pages'].fillna('0')

In [95]:
def parse_string(x):
    if '-' in x:
        pradzia, pabaiga = x.split('-')
        try: 
            return int(pabaiga) - int(pradzia)
        except ValueError:
            return np.nan
    else:
        return np.nan
def puslapiai(x):
    # listas
    if isinstance(x, list):
        stringas = x[0]
        return parse_string(stringas)
    if isinstance(x, str):
        return parse_string(x)    
    else: 
        return np.nan

In [96]:
data1['pages'] = data1['pages'].apply(lambda x: ''.join(map(str, x)))

In [97]:
data1['pages'] = data1['pages'].replace(['p'], '0')

In [98]:
apimtis = data1['pages'].str.split('-', expand=True)

In [99]:
apimtis[0] = apimtis[0].fillna('0')

In [100]:
apimtis[0] = apimtis[0].replace([''], '0')

In [101]:
apimtis[1] = apimtis[1].fillna('0')

In [102]:
publikacijos_apimtis = apimtis[1].astype(int) - apimtis[0].astype(int)

In [103]:
data1['volume_in_pages'] = publikacijos_apimtis

### Sukuriamas vienas leidinio numerio stulpelis

#### Leidinio numeri sudaro labai skirtingi irasai, nezinau, kaip juos paversti i int

In [104]:
data1['journal_number'] = data1['journal_number'].fillna('0')

In [105]:
data1['journal_number'] = data1['journal_number'].replace([''], '0')

In [106]:
data1['journal_number'] = data1['journal_number'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(str).apply(lambda x: x.replace('.0',''))

In [107]:
data1['Leidinio_numeris_1'] = data1['Leidinio_numeris_1'].fillna('0')

In [108]:
data1['Leidinio_numeris_1'] = data1['Leidinio_numeris_1'].replace([''], '0')

In [109]:
data1['Leidinio_numeris_1'] = data1['Leidinio_numeris_1'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(str).apply(lambda x: x.replace('.0',''))

In [110]:
data1["journal_number"] = data1["journal_number"].fillna(data1["Leidinio_numeris_1"])

In [111]:
data1['journal_number'] = data1['journal_number'].replace(['0'], np.nan)

In [112]:
data1 = data1.drop(['Leidinio_numeris_1'], axis=1)

## scopus_info

### Stulpelis išskaidomas į dalis

#### sukuriamas scopus_sources stulpelis su metais

In [113]:
scopus_info = data1['scopus_info'].str.split('(', expand=True)

In [114]:
scopus_sources = scopus_info[1].str.split(')', expand=True)

In [115]:
scopus_sources = scopus_sources[0].str.split(',', expand=True)

In [116]:
data1['scopus_sources'] = scopus_sources[0].fillna(0).astype(int)

#### scopus_info skaitinės vertės paverčiamos į float objektus

In [117]:
temp = scopus_info[0].str.split(' ', expand=True)

In [118]:
sarasas = [1, 3, 5]

In [119]:
for column in sarasas:
    temp[column] = temp[column].str[:-1].fillna(0)
    temp[column] = pd.to_numeric([str(x).replace(',','.') for x in temp[column]])

#### Sukuriami atskiri rodilių, ketvirčių ir metų stulpeliai ir perkeliami į pagrindinę DataFrame

In [120]:
temp['SNIP'] = np.nan
temp['SNIP_2'] = np.nan
temp['CiteScore'] = np.nan
temp['SJR'] = np.nan
temp['SJR_2'] = np.nan

In [121]:
temp.loc[temp[0] == 'SNIP:', 'SNIP'] = temp[1]
temp.loc[temp[2] == 'SNIP:', 'SNIP_2'] = temp[3]
temp.loc[temp[0] == 'CiteScore:', 'CiteScore'] = temp[1]
temp.loc[temp[2] == 'SJR:', 'SJR'] = temp[3]
temp.loc[temp[4] == 'SJR:', 'SJR'] = temp[5]

In [122]:
temp['quartile_in_caterory'] = temp[6]
temp['scopus_sources'] = scopus_sources[0]

In [123]:
temp["SNIP"] = temp["SNIP"].fillna(temp["SNIP_2"])
temp["SJR"] = temp["SJR"].fillna(temp["SJR_2"])
temp["quartile_in_caterory"] = temp["quartile_in_caterory"].fillna(np.nan)

In [124]:
data1['SNIP'] = temp['SNIP']
data1['CiteScore'] = temp['CiteScore']
data1['SJR'] = temp['SJR']
data1['quartile_in_caterory'] = temp['quartile_in_caterory']
data1['scopus_sources'] = temp['scopus_sources']

In [125]:
data1 = data1.drop(['scopus_info'], axis=1)

## if_info

### Stulpelis išskaidomas į dalis

#### sukuriamas if_info stulpelis su metais

In [126]:
if_info = data1['if_info'].str.split('(', expand=True)

#### Dalis eiluciu turi dvigubus irasus. Patikrinama, koks eiluciu skaicius yra tokio tipo:

In [127]:
print(((if_info[1].str.contains('; IF: ').value_counts().get(key = True)) / (if_info[1].count())).round(2)*100, '%')

4.0 %


#### Kadangi tokių eilučių yra 4%, papildomi įrašai nenagarinėjami

In [128]:
if_info_years = if_info[1].str.split(', ', expand=True)

In [129]:
InCites = if_info_years[1].str.split(')', expand=True)

In [130]:
InCites = InCites[0].str.split(' ', expand=True)

#### if_info skaitinės vertės paverčiamos į float objektus

In [131]:
temp_1 = if_info[0].str.split(' ', expand=True)

In [132]:
sarasas = [1, 3]

In [133]:
for column in sarasas:
    temp_1[column] = temp_1[column].str[:-1].fillna(0)
    temp_1[column] = pd.to_numeric([str(x).replace(',','.') for x in temp_1[column]])

#### Sukuriami atskiri rodilių, ketvirčių ir metų stulpeliai ir perkeliami į pagrindinę DataFrame

In [134]:
temp_1['IF'] = np.nan
temp_1['AIF'] = np.nan

In [135]:
temp_1.loc[temp_1[0] == 'IF:', 'IF'] = temp_1[1]
temp_1.loc[temp_1[2] == 'AIF:', 'AIF'] = temp_1[3]

In [136]:
temp_1['quartile_in_caterory_2'] = temp_1[4]
temp_1['if_info_years'] = if_info_years[0]
temp_1['InCites_JCR'] = InCites[2]

In [137]:
data1['IF'] = temp_1['IF']
data1['AIF'] = temp_1['AIF']
data1['quartile_in_caterory_2'] = temp_1['quartile_in_caterory_2']
data1['if_info_years'] = temp_1['if_info_years']
data1['InCites_JCR'] = temp_1['InCites_JCR']

In [138]:
data1 = data1.drop(['if_info'], axis=1)

## dbases

### Stulpelis išskaidomas į dalis

In [139]:
dbases = data1['dbases'].str.split('; ', expand=True)

In [140]:
data1['dbases_count'] = dbases.count(1)

In [141]:
dbases = pd.concat([dbases[col] for col in dbases.columns]).value_counts().to_frame()

In [142]:
dbases.head(10)

,0
Scopus,2736
Science Citation Index Expanded (Web of Science),2427
ICONDA,1346
IndexCopernicus,1291
INSPEC,1221
Academic Search Complete,851
DOAJ,657
Social Sciences Citation Index (Web of Science),630
VINITI,556
Business Source Complete,529


#### Išskaidžius data bases stulpelį buvo gautos 340 skirtingų duomenų bazių, kuriose saugomos publikacijos. Dalis iš jų pasikartoja tik kelis kartus. Dėl to buvo priimtas sprendimas atskirai identifikuoti pirmąsias 10, o likusias priskirti kategorijai "other_dbases"

In [143]:
data1['Scopus'] = data1['dbases'].str.contains('Scopus').fillna(0).astype(int)
data1['SCIE'] = data1['dbases'].str.contains('Science Citation Index Expanded').fillna(0).astype(int)
data1['ICONDA'] = data1['dbases'].str.contains('ICONDA').fillna(0).astype(int)
data1['IndexCopernicus'] = data1['dbases'].str.contains('IndexCopernicus').fillna(0).astype(int)
data1['INSPEC'] = data1['dbases'].str.contains('INSPEC').fillna(0).astype(int)
data1['Academic_Search_Complete'] = data1['dbases'].str.contains('Academic Search Complete').fillna(0).astype(int)
data1['DOAJ'] = data1['dbases'].str.contains('DOAJ').fillna(0).astype(int)
data1['SSCI'] = data1['dbases'].str.contains('Social Sciences Citation Index').fillna(0).astype(int)
data1['VINITI'] = data1['dbases'].str.contains('VINITI').fillna(0).astype(int)
data1['Business_Source_Complete'] = data1['dbases'].str.contains('Business Source Complete').fillna(0).astype(int)

In [144]:
data1['other_dbases'] = 0

In [145]:
data1['other_dbases'] = data1['dbases_count']-data1['Scopus']-data1['SCIE']-data1['ICONDA']-data1['IndexCopernicus']-data1['INSPEC']-data1['Academic_Search_Complete']-data1['DOAJ']-data1['SSCI']-data1['VINITI']-data1['Business_Source_Complete']

In [146]:
data1 = data1.drop(['dbases'], axis=1)

## res_fields / mokslo ir meno sritys

### Stulpelis išskaidomas į dalis

In [147]:
res_fields = data1['res_fields'].str.split('; ', expand=True)

In [148]:
res_fields.fillna(np.nan)

,0,1,2,3,4
0,S 003,NaN,NaN,NaN,NaN
1,T 004,NaN,NaN,NaN,NaN
2,T 004,NaN,NaN,NaN,NaN
3,S 003,S 004,NaN,NaN,NaN
4,S 004,NaN,NaN,NaN,NaN
...,...,...,...,...,...
7508,A 004,NaN,NaN,NaN,NaN
7509,H 003,NaN,NaN,NaN,NaN
7510,H 003,NaN,NaN,NaN,NaN
7511,N 002,NaN,NaN,NaN,NaN


In [149]:
res_fields[0] = res_fields[0].replace('0', np.nan)
res_fields[0] = res_fields[0].replace('00', np.nan)

#### res_fields išskaidomas į dalis ir matoma, kad publikacijų, kurios būtų priskirtos 4 arba 5 mokslo sritims yra labai mažai: 0.3 % ir 0.01 % atitinkamai. Dėl to priimamas sprendimas nagrinėti tik pirmasias 3 mokslo sritis

In [150]:
print(((res_fields[4].count()) / (res_fields[0].count())).round(4)*100, '%')

0.01 %


In [151]:
print(((res_fields[3].count()) / (res_fields[0].count())).round(3)*100, '%')

0.3 %


In [152]:
print(((res_fields[2].count()) / (res_fields[0].count())).round(3)*100, '%')

9.6 %


In [153]:
data1['res_fields_1'] = res_fields[0]
data1['res_fields_2'] = res_fields[1]
data1['res_fields_3'] = res_fields[2]

In [154]:
data1 = data1.drop(['res_fields'], axis=1)

In [155]:
data1 = data1.drop(['projection_apa'], axis=1)

## Leidiniu pavadinimu sutvarkymas

In [156]:
journals = data1['journal_name']

#### Patikrinama, kiek yra unikalių žurnalų pavadinimų

In [157]:
unikalus_pavadinimai_skaicius = journals.str.strip().nunique()

In [158]:
unikalus_pavadinimai_skaicius

1712

In [159]:
unikalus_pavadinimai = sorted(journals.astype(str).unique().tolist())

#### https://towardsdatascience.com/fuzzywuzzy-find-similar-strings-within-one-column-in-a-pandas-data-frame-99f6c2a0c212

#### Patikrinami 2 pavadinimų panašumo radimo būdai: Token Sort Ratio ir Token Set Ratio

In [160]:
process.extract('Chemical and Process Engineering', unikalus_pavadinimai, scorer=fuzz.token_sort_ratio)

[('Chemical and Process Engineering', 100),
 ('Chemical and process engineering', 100),
 ('Chemical engineering research and design', 78),
 ('Chemical engineering science', 77),
 ('Ecological Chemistry and Engineering S', 74)]

In [161]:
process.extract('Chemical and Process Engineering', unikalus_pavadinimai, scorer=fuzz.token_set_ratio)

[('Chemical and Process Engineering', 100),
 ('Chemical and Process Engineering= Inzynieria Chemiczna i Procesowa', 100),
 ('Chemical and process engineering', 100),
 ('Chemical engineering research and design', 86),
 ('Chemical engineering science', 83)]

##### Palyginus keletą atsitiktinių pavadinimų pastebėta, kad Token Set Ratio pateikia tikslesnius rezultatus. Tačiau nepaisant to, tolimesnis modelis bus kuriamas pritaikant abu būdus, kad būtų galima palyginti galutinius rezultatus

#### Pirma sukuriamas Token Sort Ratio modelis

In [162]:
#Create tuples of journals names, matched journals names, and the score
score_sort = [(x,) + i
             for x in unikalus_pavadinimai 
             for i in process.extract(x, unikalus_pavadinimai,
                                      scorer=fuzz.token_sort_ratio)]
#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, 
                               columns=['journals_sort','match_sort','score_sort'])


In [163]:
len(similarity_sort['journals_sort'])

8565

#### Kadangi pavadinimų panašumai ieškomi viename stulpelyje, gaunami dublikuoti rezultatai: Indian Journal of Physics	– Indian journal of physics ir Indian journal of physics – Indian Journal of Physics. Kad juos būtų galima pašalinti vėliau, sukuriamas papildomas stulpelis

In [164]:
similarity_sort['sorted_journals_sort'] = np.minimum(similarity_sort['journals_sort'], similarity_sort['match_sort'])
similarity_sort.head()

,journals_sort,match_sort,score_sort,sorted_journals_sort
0,14-th German-Lithuanian-Polish Colloquium on „...,14-th German-Lithuanian-Polish Colloquium on „...,100,14-th German-Lithuanian-Polish Colloquium on „...
1,14-th German-Lithuanian-Polish Colloquium on „...,"Management and Engineering'10: VIII ne , 2010 ...",45,14-th German-Lithuanian-Polish Colloquium on „...
2,14-th German-Lithuanian-Polish Colloquium on „...,"Management and Engineering'11: IX ne , 2011 lg...",45,14-th German-Lithuanian-Polish Colloquium on „...
3,14-th German-Lithuanian-Polish Colloquium on „...,Applied mechanics and materials: innovative te...,43,14-th German-Lithuanian-Polish Colloquium on „...
4,14-th German-Lithuanian-Polish Colloquium on „...,IEEE Transactions on Plasma Science. 15th Inte...,43,14-th German-Lithuanian-Polish Colloquium on „...


In [165]:
len(similarity_sort['journals_sort'])

8565

#### Išvaloma DataFrame: pasirenkamas score_sort >= 95, panaikinamos eilutės, kuriose atitikmuo yra tas pats pavadinimas ir panaikinami dublikatai

In [166]:
high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 95) &
                (similarity_sort['journals_sort'] !=  similarity_sort['match_sort']) &
                (similarity_sort['sorted_journals_sort'] != similarity_sort['match_sort'])]
high_score_sort = high_score_sort.drop('sorted_journals_sort',axis=1).copy()

#### Rezultatai pateikiami DataFrame formatu

In [167]:
sugrupuoti_pavadinimai_sort = high_score_sort.groupby(['journals_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)

In [168]:
sugrupuoti_pavadinimai_sort

,,match_sort
journals_sort,score_sort,
ACI Structural Journal,100,ACI structural journal
Journal of Radioanalytical and Nuclear Chemistry,100,Journal of radioanalytical and nuclear chemistry
Journal of molecular catalysis B: Enzymatic,100,Journal of molecular catalysis. B: Enzymatic
KSI Transactions on Knowledge Society: a publication of the Knowledge Society Institute,100,KSI transactions on knowledge society: a publi...
"Kraštovaizdžio architektūra - profesijos horizontai ir sinergija: Kraštovaizdžio architektūros forumo 2015, vykusio Vilniuje 2015 m. spalio 29 d., mokslo darbai",100,Kraštovaizdžio architektūra - profesijos horiz...
...,...,...
Economic computation and economic cybernetics studies and research,95,Economic computation and economic cybernetics ...
Technical Physics Letters: an English translation of the journal ''Zhurnal Tekhnicheskoi Fiziki'',95,Technical physics letters: an English translat...
Mathematic,95,Mathematics


In [169]:
len(sugrupuoti_pavadinimai_sort)

220

#### Gauta 220 pavadinimų porų, kurių atitikmuo yra >= 95

#### Visa žingsnių eiga pakartojama Token Set Ratio modeliui sukurti

In [170]:
#Create tuples of brand names, matched brand names, and the score
score_set = [(x,) + i
             for x in unikalus_pavadinimai 
             for i in process.extract(x, unikalus_pavadinimai,     scorer=fuzz.token_set_ratio)]
#Create a dataframe from the tuples
similarity_set = pd.DataFrame(score_set, columns=['journals_set','match_set','score_set'])

In [171]:
len(similarity_set['journals_set'])

8565

In [172]:
similarity_set['sorted_journals_set'] = np.minimum(similarity_set['journals_set'], similarity_set['match_set'])
similarity_set.head()

,journals_set,match_set,score_set,sorted_journals_set
0,14-th German-Lithuanian-Polish Colloquium on „...,14-th German-Lithuanian-Polish Colloquium on „...,100,14-th German-Lithuanian-Polish Colloquium on „...
1,14-th German-Lithuanian-Polish Colloquium on „...,chnology and Management in Construction,87,14-th German-Lithuanian-Polish Colloquium on „...
2,14-th German-Lithuanian-Polish Colloquium on „...,GPS Solutions,82,14-th German-Lithuanian-Polish Colloquium on „...
3,14-th German-Lithuanian-Polish Colloquium on „...,GPS solutions,82,14-th German-Lithuanian-Polish Colloquium on „...
4,14-th German-Lithuanian-Polish Colloquium on „...,Construction Science,75,14-th German-Lithuanian-Polish Colloquium on „...


In [173]:
len(similarity_set['journals_set'])

8565

In [174]:
high_score_set = similarity_set[(similarity_set['score_set'] >= 95) &
                (similarity_set['journals_set'] !=  similarity_set['match_set']) &
                (similarity_set['sorted_journals_set'] != similarity_set['match_set'])]
high_score_set = high_score_set.drop('sorted_journals_set',axis=1).copy()

In [175]:
sugrupuoti_pavadinimai_set = high_score_set.groupby(['journals_set','score_set']).agg(
                        {'match_set': ', '.join}).sort_values(
                        ['score_set'], ascending=False)

In [176]:
len(sugrupuoti_pavadinimai_set)

505

#### Gauta 505 pavadinimų porų, kurių atitikmuo yra >= 95

#### Kadangi Token Set Ratio atveju gauta ženkliai daugiau atitikmenų, šis modelis pasirenkamas kaip toliau naudotinas darbe

#### Sukuriama nauja DataFrame, kurioje kiekvienam leidiniui suteikiamas indeksas

In [177]:
journals_set = pd.DataFrame(high_score_set['journals_set'].unique())

In [178]:
journals_set['index'] = np.arange(journals_set.shape[0])
journals_set = journals_set.rename(columns = {0:'journal'})

#### Šis indeksas pridedamas prie sugrupuotų leidinių duomenų lentelės

In [179]:
high_score_set_index = pd.merge(high_score_set, journals_set, how='left' ,left_on='journals_set', right_on='journal')

In [180]:
high_score_set_index = high_score_set_index.drop(['journal'], axis=1)

#### Buvo pastebėta, kad leidiniai lentelėje kartojasi: vienu atveju jie yra kaip pirmas leidinys, kuriam priskirti kiti, kitu atveju - jis yra priskirtas kitiems leidiniams. Deja, bet šis priskyrimas nesutampta 100%, todėl rezultate vienas leidinys gali turėti kelis indeksus. Tam panaikinti, atskiriama antroji lentelės dalis su indeksais.

In [181]:
index_check = high_score_set_index.drop(['journals_set', 'score_set'], axis=1)

In [182]:
index_check = index_check.rename(columns = {'match_set':'match_journal', 'index':'second_index'})

In [183]:
index_check = index_check.drop_duplicates(subset ="match_journal")

#### Toliau abi lentelės sujungiamos ir gaunamos 2 indeksų vertės: viena pirmoji, kita priskirta vėliau. Tikslas jas palyginti ir suvienodinti

In [184]:
high_score_set_index_2 = pd.merge(high_score_set_index, index_check, how='left',
                                  left_on='journals_set', right_on='match_journal')

#### Sukuriamas papildomas stulpelis su mažesne indekso verte

In [185]:
high_score_set_index_2['final_index'] = high_score_set_index_2[['index','second_index']].min(axis=1)

#### Sukuriama atrskira DataFrame, kurioje būtų tik leidiniai ir jų galutiniai indeksai

In [186]:
leidiniu_indeksai = high_score_set_index_2.drop(['score_set', 'index', 'match_journal', 'second_index'], axis=1)

In [187]:
leidiniu_indeksai_1 = leidiniu_indeksai.drop(['match_set'], axis=1)

In [188]:
leidiniu_indeksai_2 = leidiniu_indeksai.drop(['journals_set'], axis=1)

In [189]:
leidiniu_indeksai_1.columns = leidiniu_indeksai_2.columns
leidiniu_indeksai_visi = pd.concat([leidiniu_indeksai_1, leidiniu_indeksai_2], axis=0, ignore_index=True)

In [190]:
leidiniu_indeksai_visi = leidiniu_indeksai_visi.drop_duplicates()

In [191]:
leidiniu_indeksai_visi = leidiniu_indeksai_visi.sort_values(by=['match_set', 'final_index'])

In [192]:
leidiniu_indeksai_visi = leidiniu_indeksai_visi.drop_duplicates(subset ="match_set")

#### Leidinių indeksai perkeliami į pagrindinę duomenų lentelę

In [193]:
data1 = pd.merge(data1, leidiniu_indeksai_visi, how='left' ,left_on='journal_name', right_on='match_set')

#### Iš pagrindinės duomenų lentelės išfiltruojami leidiniai, kurie nebuvo priskirti prie tų, kurie yra su panašiomis vertėmis. Jiems suteikiami atskiri indeksai

In [194]:
missing_index = data1[data1['final_index'].isnull()]

In [195]:
missing_index = missing_index[['journal_name', 'final_index']].drop_duplicates(subset ="journal_name").sort_values(by=['journal_name'])

In [196]:
missing_index = missing_index.drop_duplicates(subset ="journal_name")

In [197]:
missing_index['index'] = np.arange(missing_index.shape[0])

In [198]:
missing_index = missing_index.drop(['final_index'], axis=1)

In [199]:
missing_index['final_index_2'] = missing_index['index']+max(leidiniu_indeksai_visi['final_index'])+1

In [200]:
missing_index = missing_index.drop(['index'], axis=1)

#### Pavienių leidinių indeksai perkeliami į pagrindinę duomenų lentelę

In [201]:
data1 = pd.merge(data1, missing_index, how='left' ,left_on='journal_name', right_on='journal_name')

In [202]:
data1["final_index"] = data1["final_index"].fillna(data1["final_index_2"])

In [203]:
data1 = data1.drop(['final_index_2', 'match_set'], axis=1)

#### Originaliai buvusių skirtingų leidinių pavadinimų ir sugrūpuotų santykis

In [204]:
data1['journal_name'].unique().size

1713

In [205]:
data1['final_index'].unique().size

1254

In [210]:
(data1['final_index'].unique().size / data1['journal_name'].unique().size)

0.7320490367775832

## Leidinių ir publikacijų duomenų rinkinių sukūrimas

### Sukurtos DataFrame perkeliamos į csv. failus ir išsaugomos prie duomenų, kad būtų naudojamos tolimesniuose procesuose

In [211]:
publikacijos = data1.drop(['journal_number', 
                           'journal_name_in_foreign_language_1', 
                           'journal_name_in_foreign_language_2', 
                           'journal_name_in_foreign_language_3',
                           'scopus_sources',
                           'SNIP',
                           'CiteScore',
                           'SJR',
                           'quartile_in_caterory',
                           'IF',
                           'AIF',
                           'quartile_in_caterory_2',
                           'if_info_years',
                           'InCites_JCR'], axis=1)

In [212]:
leidiniai = data1.drop(['docs_id',
                        'doc_code',
                        'pub_year',
                        'doi',
                        'pages',
                        'publication',
                        'volume_in_pages',
                        'dbases_count',
                        'Scopus',
                        'SCIE',
                        'ICONDA',
                        'IndexCopernicus',
                        'INSPEC',
                        'Academic_Search_Complete',
                        'DOAJ',
                        'SSCI',
                        'VINITI',
                        'Business_Source_Complete',
                        'other_dbases',
                        'res_fields_1',
                        'res_fields_2',
                        'res_fields_3'], axis=1)

In [213]:
#publikacijos.to_csv('publikacijos.csv')

In [214]:
#leidiniai.to_csv('leidiniai.csv')

## Autoriai 

#### Įkeliamas autorių failas, kuriame autorių vardai pakeisti į indeksus. Šis procesas buvo atliktas Excel programoje:


– Autorių pavardės peržiūrėtos ir suvienodintos \
– Pašalinti dublikatai \
– Sukurti indeksai 

In [215]:
authors = pd.read_csv('authors.csv')

#### Apskaičiuojama, kiek kartų kiekvienas autorius pasikartojo duomenų rinkinyje

In [216]:
authors_count = pd.concat([authors[col] for col in authors.columns]).value_counts().to_frame()

### Autorių bendraautorystės sąrašo sudarymas

#### Nan reikšmės pakeičiamos į 0

In [217]:
autoriu_sarasas = authors.fillna(0).values.astype(int).tolist()

#### Pašalinami visi 0 iš autorių sąrašų

In [218]:
bendraautoriai_visi = [[ele for ele in sub if ele != 0] for sub in autoriu_sarasas]

#### Pašalinami visi tušti sąrašai, kurie reiškia, kad publikaciją parašė vienas autorius (nėra bendraautorystės)

In [219]:
bendraautoriai_poros = [[ele for ele in sub if len(sub) != 1] for sub in bendraautoriai_visi]

In [220]:
bendraautoriai = list(filter(lambda x: len(x) != 0, bendraautoriai_poros))

#### Sąrašai išrikiuojami didėjimo tvarka

In [221]:
bendraautoriai_1 = [sorted(i) for i in bendraautoriai]

#### Gaunamos visos unikalios autoriu poros

In [222]:
poros = [list(itertools.combinations(i,2)) for i in bendraautoriai_1]

#### Poros + svoriai

In [223]:
autoriai_dataframe = pd.DataFrame([i for j in poros for i in j])

In [224]:
edges = autoriai_dataframe.groupby(autoriai_dataframe.columns.tolist(),as_index=False).size()

In [225]:
edges = edges.drop(edges[edges[0] == edges[1]].index)

In [226]:
edges

,0,1,size
0,1,858,1
1,1,1334,1
2,1,1728,1
3,2,8,3
4,2,17,3
...,...,...,...
27651,5511,5519,2
27652,5525,5540,1
27653,5541,5545,3
27654,5542,5546,1


#### Apskaičiuojama, kiek yra unikalių bendraautorystės porų

In [227]:
edges.groupby([0, 1]).ngroups

27633

#### Gauta DataFrame išsaugoma kaip atskiras failas, kuris, naudojamas grafo kūrimui. Pridedamas papildomas stulpelis, kuriame nurodoma, kad grafas nėra orientuotas (Undirected), nes bendraautorystė yra lygi abiejomis kryptimis

## Autoriai IKI 20 BENDRADARBIAVIMO

#### Kuriamas analogiškas autorių bendradarbiavimo sąrašas, tačiau jame pašalintos išskirtys: 6 publikacijos, kurių autorių skaičius >= 20

#### Įkeliamas autorių failas, kuriame autorių vardai pakeisti į indeksus

In [228]:
authors_adj = pd.read_csv('authors_adjusted.csv')

#### Apskaičiuojama, kiek kartų kiekvienas autorius pasikartojo duomenų rinkinyje

In [229]:
authors_count_adj = pd.concat([authors_adj[col] for col in authors_adj.columns]).value_counts().to_frame()

#### Nan reikšmės pakeičiamos į 0

In [230]:
autoriu_sarasas_adj = authors_adj.fillna(0).values.astype(int).tolist()

#### Pašalinami visi 0 iš autorių sąrašų

In [231]:
bendraautoriai_visi_adj = [[ele for ele in sub if ele != 0] for sub in autoriu_sarasas_adj]

#### Pašalinami visi tušti sąrašai, kurie reiškia, kad publikaciją parašė vienas autorius (nėra bendraautorystės)

In [232]:
bendraautoriai_poros_adj = [[ele for ele in sub if len(sub) != 1] for sub in bendraautoriai_visi_adj]

In [233]:
bendraautoriai_adj = list(filter(lambda x: len(x) != 0, bendraautoriai_poros_adj))

#### Sąrašai išrikiuojami didėjimo tvarka

In [234]:
bendraautoriai_1_adj = [sorted(i) for i in bendraautoriai_adj]

#### Gaunamos visos unikalios autoriu poros

In [235]:
poros_adj = [list(itertools.combinations(i,2)) for i in bendraautoriai_1_adj]

#### Poros + svoriai

In [236]:
autoriai_dataframe_adj = pd.DataFrame([i for j in poros_adj for i in j])

In [237]:
edges_adj = autoriai_dataframe_adj.groupby(autoriai_dataframe_adj.columns.tolist(),as_index=False).size()

In [238]:
edges_adj = edges_adj.drop(edges_adj[edges_adj[0] == edges_adj[1]].index)

#### Apskaičiuojama, kiek yra unikalių bendraautorystės porų

In [239]:
edges_adj.groupby([0, 1]).ngroups

19275